In [71]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print("adding project dir to path:", repo_dir)
if repo_dir not in sys.path:
    sys.path = [repo_dir] + sys.path
else:
    sys.path.remove(repo_dir)
    sys.path = [repo_dir] + sys.path
    


adding project dir to path: /Users/marijnkoolen/Code/Huygens/republic-project


## Reading the IOB-formatted files

In [108]:
import glob
import os
from collections import defaultdict

# this is the folder in which train, test and dev files will be saved
data_folder = f'{repo_dir}/ground_truth/entities/tag_de_besluiten/tdb-data'
train_folder = f'{repo_dir}/ground_truth/entities/tag_de_besluiten/flair_training'

layer_files = defaultdict(list)
res_ids = set()

tagged_res_files = glob.glob(os.path.join(data_folder, 'session**/*.tsv'))
for tagged_res_file in tagged_res_files:
    fname_parts = tagged_res_file.split('/')
    layer_name = fname_parts[-1].replace('.tsv', '')
    res_id = fname_parts[-2].replace('.txt', '')
    if layer_name == 'all_tags':
        continue
    else:
        layer_name = layer_name.replace('tag_', '')
    layer_files[layer_name].append({'filename': tagged_res_file, 'res_id': res_id})
    res_ids.add(res_id)

In [109]:
res_ids = list(res_ids)
res_ids

['session-1785-05-02-ordinaris-num-1-resolution-22',
 'session-1720-01-22-ordinaris-num-1-resolution-8',
 'session-1706-10-13-ordinaris-num-1-resolution-20',
 'session-1787-03-26-ordinaris-num-1-resolution-7',
 'session-1706-08-21-ordinaris-num-1-resolution-20',
 'session-1793-03-29-ordinaris-num-1-resolution-12',
 'session-1709-04-02-ordinaris-num-1-resolution-15',
 'session-1759-07-30-ordinaris-num-1-resolution-11',
 'session-1705-07-27-ordinaris-num-1-resolution-2',
 'session-1724-07-27-ordinaris-num-1-resolution-1',
 'session-1725-10-09-ordinaris-num-1-resolution-9',
 'session-1717-03-12-ordinaris-num-1-resolution-9',
 'session-1720-05-18-ordinaris-num-1-resolution-8',
 'session-1717-09-18-ordinaris-num-1-resolution-5',
 'session-1755-07-31-ordinaris-num-1-resolution-1',
 'session-1729-08-12-ordinaris-num-1-resolution-4',
 'session-1768-11-07-ordinaris-num-1-resolution-13',
 'session-1728-02-17-ordinaris-num-1-resolution-6',
 'session-1713-10-25-ordinaris-num-1-resolution-13',
 'se

## Creating the Train, Validate and Test files

Randomly distributing the content of the IOB-formatted files, using an 80-10-10 split.

In [111]:
random_seed = 89234
random.seed(random_seed)

random.shuffle(res_ids)

file_set_res_ids = {
    'test': [res_id for res_id in res_ids[:163]],
    'validate': [res_id for res_id in res_ids[163:2*163]],
    'train': [res_id for res_id in res_ids[2*163:]]
}

len(file_set_res_ids['test']), len(file_set_res_ids['validate']), len(file_set_res_ids['train'])


(163, 163, 1305)

In [121]:
def write_output_file(output_file, input_files):
    with open(output_file, 'wt') as fh_out:
        for input_file in input_files:
            with open(input_file, 'rt') as fh_in:
                for line in fh_in:
                    fh_out.write(line.replace('\t', ' '))
                fh_out.write('\n')
    return None


for layer_name in layer_files:
    layer_dir = f'{repo_dir}/ground_truth/entities/tag_de_besluiten/flair_training_{layer_name}'
    if os.path.exists(layer_dir) is False:
        os.mkdir(layer_dir)
    for file_set in ['validate', 'test']:
        set_files = [lf['filename'] for lf in layer_files[layer_name] if lf['res_id'] in file_set_res_ids[file_set]]
        print(file_set, layer_name, len(layer_files[layer_name]), len(set_files))
        output_file = os.path.join(layer_dir, f'{file_set}.txt')
        write_output_file(output_file, set_files)
        print(output_file)
    train_files = [lf['filename'] for lf in layer_files[layer_name] if lf['res_id'] in file_set_res_ids['train']]
    num_train_files = len(train_files)
    for train_size in [i / 10 for i in range(1,11)]:
        num_set_files = int(num_train_files * train_size)
        set_files = train_files[:num_set_files]
        print(file_set, layer_name, len(layer_files[layer_name]), len(set_files))
        output_file = os.path.join(layer_dir, f'train_{train_size}.txt')
        write_output_file(output_file, set_files)
        print(output_file)

    

validate RES 1631 163
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/validate.txt
test RES 1631 163
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/test.txt
test RES 1631 130
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/train_0.1.txt
test RES 1631 261
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/train_0.2.txt
test RES 1631 391
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/train_0.3.txt
test RES 1631 522
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/train_0.4.txt
test RES 1631 652
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_RES/train_0.5.txt
test RES 1631 783
/Use

/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_NAM/train_0.6.txt
test NAM 1631 913
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_NAM/train_0.7.txt
test NAM 1631 1044
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_NAM/train_0.8.txt
test NAM 1631 1174
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_NAM/train_0.9.txt
test NAM 1631 1305
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_NAM/train_1.0.txt
validate COM 1631 163
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_COM/validate.txt
test COM 1631 163
/Users/marijnkoolen/Code/Huygens/republic-project/ground_truth/entities/tag_de_besluiten/flair_training_COM/test.txt
test COM 1631 130
/Users/marijnkoolen